In [2]:
import numpy as np
from sklearn.datasets import fetch_openml 
mnist = fetch_openml('mnist_784', version=1)

In [3]:
print((np.array(mnist.data.loc[42]).reshape(28, 28) > 0).astype(int))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 

In [4]:
X, y = mnist["data"], mnist["target"].astype(np.uint8)
y = y.sort_values()
X = X.reindex(y.index)
X_train, X_test = X[:56000], X[56000:]
y_train, y_test = y[:56000], y[56000:]
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(56000, 784) (56000,)
(14000, 784) (14000,)


In [5]:
y_train.unique()

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=uint8)

In [6]:
y_test.unique()

array([7, 8, 9], dtype=uint8)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(56000, 784) (56000,)
(14000, 784) (14000,)


In [8]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [9]:
np.unique(y_test)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [10]:
y_train_0 = (y_train == 0)
y_test_0 = (y_test == 0)
print(y_train_0)
print(np.unique(y_train_0))
print(len(y_train_0))

32576    False
10306    False
37102    False
57835    False
15160     True
         ...  
57853    False
3896      True
68497    False
20351     True
36463    False
Name: class, Length: 56000, dtype: bool
[False  True]
56000


In [11]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42) 
sgd_clf.fit(X_train, y_train_0)

SGDClassifier(random_state=42)

In [12]:
print(sgd_clf.predict([mnist["data"].loc[0], mnist["data"].loc[1]]))

[False  True]


c:\users\aneta_p\documents\studia\semestr_4\machine_learning\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


In [13]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

y_train_pred = sgd_clf.predict(X_train)
y_test_pred = sgd_clf.predict(X_test)

accuracy = [accuracy_score(y_train_0, y_train_pred), accuracy_score(y_test_0, y_test_pred)]

In [14]:
print(accuracy)

[0.9832142857142857, 0.9818571428571429]


In [16]:
import pickle
with open('sgd_acc.pkl', 'wb') as f:
    pickle.dump(accuracy, f)

In [17]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(sgd_clf, X_train, y_train_0,
                        cv=3, scoring="accuracy",
                        n_jobs=-1)
print(score)

[0.98842878 0.98805378 0.98799957]


In [18]:
sgd_clf.predict(mnist["data"])

array([False,  True, False, ..., False, False, False])

In [19]:
with open('sgd_cva.pkl', 'wb') as f:
    pickle.dump(score, f)

In [22]:
sgd_m_clf = SGDClassifier(random_state=42,n_jobs=-1)
sgd_m_clf.fit(X_train, y_train)

SGDClassifier(n_jobs=-1, random_state=42)

In [23]:
print(sgd_m_clf.predict([mnist["data"].loc[0], mnist["data"].loc[1]]))

[5 0]


c:\users\aneta_p\documents\studia\semestr_4\machine_learning\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


In [24]:
print(cross_val_score(sgd_m_clf, X_train, y_train, cv=3, scoring="accuracy", n_jobs=-1))
y_train_pred = cross_val_predict(sgd_m_clf, X_train, y_train, cv=3, n_jobs=-1)

[0.88048428 0.87132373 0.8772099 ]


In [25]:
conf_mx = confusion_matrix(y_train, y_train_pred)
print(conf_mx)

[[5295    2   22   42   16   26   37    6   71   21]
 [   3 6098   39   19    4   18    6    9   64   25]
 [  53   74 4798  131   65   18   86   65  273   38]
 [  41   30  157 4888   12  179   24   50  260  106]
 [  22   30   30   14 4691   22   69   23  156  401]
 [  93   29   41  251   67 3806  113   20  477  106]
 [  53   28   45   10   59   63 5114    5   92    8]
 [  27   32   48   40   67   10    6 5154  108  358]
 [  49  122   67  146   32  288   35   15 4518  183]
 [  31   29   18   66  287   42    5  217  178 4713]]


In [26]:
with open('sgd_cmx.pkl', 'wb') as f:
    pickle.dump(conf_mx, f)